# Drug Consumption: Pré-processamento de Dados

In [13]:
import pandas as pd

data = pd.read_csv("data/drug_consumption.csv", index_col="ID")

# Pré-processamento dos dados
Os dados do dataset estão em valores pouco legíveis, assim, alguns processamentos foram executados para possibilitar o uso em classificadores.

## Idade - "Age"
Engloba um range de idades, entretanto, é remapeado (posteriormente) para valores discretos, mantendo a distância entre os itens.

In [14]:
age_col = {
    -0.95197: "18 - 24",
    -0.07854: "25 - 34",
    0.49788: "35 - 44",
    1.09449: "45 - 54",
    1.82213: "55 - 64",
    2.59171: "65+",
}

# Se for necessário, estão guardados valores nominais no dataset
data["Age_"] = data["Age"].replace(age_col)

## Gênero - "Gender"
Dividido entre "Female" e "Male". Aqui, é transformada de forma que a coluna indica se o indivíduo é homem ou não.
Poderia ser feito com OneHotEncoder, mas escolhemos poupar o uso de mais colunas.

In [15]:
# gender_col = {0.48246: "Female", -0.48246: "Male"}
gender_col = {0.48246: 0, -0.48246: 1}
data["Gender"] = data["Gender"].replace(gender_col)

## Escolaridade - "Education"
Indica o nível de escolaridade do indivíduo. Testamos representar de uma forma binária separando indivíduos que possuem algum diploma profissional ou não.

In [16]:
education_col = {
    -2.43591: "Left School Before 16 years",
    -1.73790: "Left School at 16 years",
    -1.43719: "Left School at 17 years",
    -1.22751: "Left School at 18 years",
    -0.61113: "Some College, No Certificate Or Degree",
    -0.05921: "Professional Certificate/Diploma",
    0.45468: "University Degree",
    1.16365: "Masters Degree",
    1.98437: "Doctorate Degree",
}

data["Education_"] = data["Education"].replace(education_col)

## País - "Country"
Indica o país de origem do indivíduo. Será mais relevante na análise de distribuição dos dados, mas pode causar algum bias desnecessário na classificação.

In [17]:
country_col = {
    -0.09765: "Australia",
    0.24923: "Canada",
    -0.46841: "New Zealand",
    -0.28519: "Other",
    0.21128: "Republic of Ireland",
    0.96082: "UK",
    -0.57009: "USA",
}
data["Country"] = data["Country"].replace(country_col)

## Etnia - "Ethnicity"
Indica a etnia do indivíduo. Pode ser mais relevante na análise dos dados, mas parece irrelevante e enviezado utilizá-la no classificador.

In [18]:
ethnicity_col = {
    -0.50212: "Asian",
    -1.10702: "Black",
    1.90725: "Mixed-Black/Asian",
    0.12600: "Mixed-White/Asian",
    -0.22166: "Mixed-White/Black",
    0.11440: "Other",
    -0.31685: "White",
}
data["Ethnicity"] = data["Ethnicity"].replace(ethnicity_col)

## Colunas das substâncias
Existem 18 substâncias presentes no dataset. Os valores apresentados nas colunas indicam qual foi a última vez que a substância foi utilizada. Nesse caso, remapeamos cada valor para um domínio discreto em que a distância de tempo é mantida.

É um range de [0, 6] que representa, respectivamente:
- (0) "nunca";
- (1) "há mais de uma década";
- (2) "na última década";
- (3) "no último ano";
- (4) "no último mês";
- (5) "na última semana" e
- (6) "no dia anterior".

In [19]:
drugs = [
    "Alcohol",
    "Amphet",
    "Amyl",
    "Benzos",
    "Caff",
    "Cannabis",
    "Choc",
    "Coke",
    "Crack",
    "Ecstasy",
    "Heroin",
    "Ketamine",
    "Legalh",
    "LSD",
    "Meth",
    "Mushrooms",
    "Nicotine",
    "Semer",
    "VSA",
]

order_ = {
    "CL0": "Never Used",
    "CL1": "Used over a Decade Ago",
    "CL2": "Used in Last Decade",
    "CL3": "Used in Last Year",
    "CL4": "Used in Last Month",
    "CL5": "Used in Last Week",
    "CL6": "Used in Last Day",
}

# Deixamos os labels originais no dataset em colunas alternativas com underline no final (caso for necessário saber o label nominal para visualizações)
data[[x + "_" for x in drugs]] = data[drugs].applymap(lambda x: order_[x])

## Medidas de Personalidade
O dataset ainda apresenta algumas métricas chamadas de "personality measurements". São elas:
- NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness)
- BIS-11 (impulsivity)
- ImpSS (sensation seeking)

In [20]:
data.rename(
    columns={
        "Nscore": "Neuroticism",
        "Escore": "Extraversion",
        "Oscore": "Openness",
        "Ascore": "Agreeableness",
        "Cscore": "Conscientiousness",
        "Impulsive": "Impulsiveness",
        "SS": "Sensationness",
    },
    inplace=True,
)

## Aplicação de OrdinalEncoder e OneHotEncoder
Aplicamos OrdinalEncoder em colunas como "Age" e as colunas de substâncias, que apresentam valores com ordem explícita e OneHotEncoder para características categóricas sem senso perceptível de distância.

In [21]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer


ordCols = ["Age", "Education"] + drugs

preprocessor = ColumnTransformer(
    [
        ("ordinal", OrdinalEncoder(dtype=int), ordCols),
    ],
    verbose_feature_names_out=False,
    remainder="passthrough",
)
preprocessor.set_output(transform="pandas")

df_out : pd.DataFrame = preprocessor.fit_transform(data)

# Poderia ser feito anteriormente, mas aplicamos o filtro binário aqui.
df_out["Education"] = df_out["Education"].apply(lambda x: 0 if x < 5 else 1)

In [22]:
display(df_out)
df_out.to_csv("data/drug_consumption_preprocessed.csv")

,Age,Education,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,...,Ecstasy_,Heroin_,Ketamine_,Legalh_,LSD_,Meth_,Mushrooms_,Nicotine_,Semer_,VSA_
ID,,,,,,,,,,,,,,,,,,,,,
1,2,1,5,2,0,2,6,0,5,0,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
2,1,1,5,2,2,0,6,4,6,3,...,Used in Last Month,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Year,Never Used,Used in Last Month,Never Used,Never Used
3,2,1,6,0,0,0,6,3,4,0,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Never Used
4,0,1,4,0,0,3,5,2,4,2,...,Never Used,Never Used,Used in Last Decade,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
5,2,1,4,1,1,0,6,3,6,0,...,Used over a Decade Ago,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Used in Last Decade,Used in Last Decade,Never Used,Never Used
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,0,0,5,0,0,0,4,5,4,0,...,Never Used,Never Used,Never Used,Used in Last Year,Used in Last Year,Never Used,Never Used,Never Used,Never Used,Used in Last Week
1885,0,0,5,0,0,0,5,3,4,0,...,Used in Last Decade,Never Used,Never Used,Used in Last Year,Used in Last Week,Used in Last Month,Used in Last Month,Used in Last Week,Never Used,Never Used
1886,1,1,4,6,5,5,6,6,6,4,...,Used in Last Month,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Day,Never Used,Never Used
